In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image, ImageOps 


size = 64, 64
#size = 256, 256
for f in os.listdir('/kaggle/input/pokemon-images-dataset/pokemon_jpg/pokemon_jpg'):
    im = Image.open('/kaggle/input/pokemon-images-dataset/pokemon_jpg/pokemon_jpg/'+f).resize(size, Image.ANTIALIAS)
    break
    
    
big_arr = np.array([np.array(im)]).reshape(1, 64, 64, 3)
print(big_arr.shape)
i = 0
for f in os.listdir('/kaggle/input/pokemon-images-dataset/pokemon_jpg/pokemon_jpg')[1:]:
    big_arr = np.append(big_arr, [np.array(Image.open('/kaggle/input/pokemon-images-dataset/pokemon_jpg/pokemon_jpg/'+f).resize(size, Image.ANTIALIAS)).reshape(64, 64, 3) ], axis=0)
    i += 1
    
arr = np.array(im)
print(arr.shape)
plt.imshow(big_arr[3])

In [ ]:
big_arr.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LeakyReLU
import tensorflow as tf
    

units = 128
bottleneck_units = 64
restoration_units = 256
regularizer_weight = 0
input_shape = [64, 64, 3]


class BottleneckPriorScale(keras.layers.Layer):
    def __init__(self, bottleneck_units, units, regularizer_weight):
        super(BottleneckPriorScale, self).__init__()
        #self.layer1 = layers.Conv2D(bottleneck_units, kernel_size=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding='same') #no activation
        #self.layer2 = layers.Conv2D(units, kernel_size=(3, 3), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")
        #activations after every conv!!!!
        
        self.layer5 = layers.Conv2D(bottleneck_units, kernel_size=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")
        self.layera = layers.LeakyReLU()
        self.layerb = layers.Dropout(.3)
        self.layerc = layers.BatchNormalization(renorm=True)
        
        self.layer8 = layers.Conv2D(units, kernel_size=(3, 3), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")
        self.layer9 = layers.LeakyReLU()
        self.layer10 = layers.Dropout(.3)
        self.layer11 = layers.BatchNormalization(renorm=True)
        self.layer12 = layers.Conv2D(bottleneck_units, kernel_size=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding='same')
        
        self.layerd = layers.LeakyReLU()
        self.layere = layers.Dropout(.3)
        self.layerf = layers.BatchNormalization(renorm=True)
        self.layer13 = layers.Conv2D(units, kernel_size=(3, 3), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")

        #self.layer16 = layers.Conv2D(bottleneck_units, kernel_size=(1, 3), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")

        #self.layer19 = layers.Conv2D(units, kernel_size=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")
        self.layer20 = layers.LeakyReLU()
        self.layer21 = layers.Dropout(.3)
        self.layer22 = layers.BatchNormalization(renorm=True)

    def call(self, inputs):
        #x = self.layer1(inputs)
        #x = self.layer2(x)

        x = self.layer5(inputs)
        x = self.layera(x)
        x = self.layerb(x)
        x = self.layerc(x)

        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        x = self.layerd(x)
        x = self.layere(x)
        x = self.layerf(x)
        
        x = self.layer13(x)

        #x = self.layer16(x)

        #x = self.layer19(x)
        x = self.layer20(x)
        x = self.layer21(x)
        x = self.layer22(x)
        
        return x

    
    
image_reconstruction_64 = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape, renorm=True),
    layers.Conv2D(units, kernel_size=(3, 3), input_shape=input_shape, padding="same"),

    
    BottleneckPriorScale(bottleneck_units, units*2, regularizer_weight),
    layers.MaxPooling2D(),
    
    #Projection!
    layers.Conv2D(units*8, kernel_size=(1, 1), padding="same"),
    layers.LeakyReLU(),
    layers.BatchNormalization(renorm=True),
    BottleneckPriorScale(bottleneck_units, units*4, regularizer_weight),
    layers.MaxPooling2D(),
    
    #Projection!
    layers.Conv2D(units*16, kernel_size=(1, 1),padding="same"),
    layers.LeakyReLU(),
    layers.BatchNormalization(renorm=True),
    BottleneckPriorScale(bottleneck_units, units*4, regularizer_weight),
    layers.MaxPooling2D(),
    
    #Projection!
    layers.Conv2D(units*16, kernel_size=(1, 1), padding="same"),
    layers.LeakyReLU(),
    layers.BatchNormalization(renorm=True),
    BottleneckPriorScale(bottleneck_units, units*8, regularizer_weight),
    layers.MaxPooling2D(),

    #Projection!
    layers.Conv2D(units*16, kernel_size=(1, 1), padding="same"),
    layers.LeakyReLU(),
    layers.BatchNormalization(renorm=True),
    BottleneckPriorScale(bottleneck_units, units*8, regularizer_weight),
    layers.MaxPooling2D(),
    
    BottleneckPriorScale(bottleneck_units, units*16, regularizer_weight),
    layers.MaxPooling2D(),
    
    #BottleneckPriorScale(bottleneck_units, units*128, regularizer_weight),
    
    #BottleneckPriorScale(bottleneck_units, units*128, regularizer_weight),

    
    #layers.Dense(units*16),
    #layers.LeakyReLU(),
    #layers.BatchNormalization(renorm=True),
    layers.Dense(units*32, name="half"),
    layers.LeakyReLU(),
    layers.BatchNormalization(renorm=True),
    layers.Dense(1024),
    layers.LeakyReLU(),
    layers.BatchNormalization(renorm=True),
    
    layers.Dense(64*64*3),
    layers.LeakyReLU(),
    layers.Reshape(input_shape, name="decoder_output")
    
   # layers.Conv2D(input_shape[-1], (1, 1), strides=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight),
   #               
   #               #activation='relu', 
   #               activation=layers.LeakyReLU(), 
   #               
   #               padding="same", name='last_layer'),

])



def reconstruction_loss_func(old, new):
    #1 for each wrong pixel, harsh!
    
    reconstruction_loss = keras.losses.binary_crossentropy(tf.bitcast(tf.cast(old, tf.int8), tf.uint8), tf.bitcast(tf.cast(new, tf.int8), tf.uint8))
    reconstruction_loss *= (256*256*3)
    
    return reconstruction_loss

def l1_loss(old, new):
    return tf.reduce_mean(tf.cast(old, tf.float32)-new)

image_reconstruction_64.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.01, clipnorm=1.0, clipvalue=.5),
    #loss=reconstruction_loss_func,
    loss='mse',
    metrics=['accuracy']
)

In [ ]:
#y = train_df.pop("label")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping



#1x1 image_restoration B4 ACTIVATION??? 256->64->256->activation
#would reduce computation (b/c I have the 3x3 do the restoring)


#Conv2DTranspose SUCKS?????
#Check!!!


    
#Reconstruction just immediately killed gradients???
#Or Dying Relu problem

#ADDED WAY MORE ACTIVATIONS
#Too many local minimum, Too many (64*64)-1 examples of 1 pixel correct all others wrong
#Similar to how I use to have all conv have activations


#BAD IDEA TO ADD FILTERS AS THE WIDTH
#THEY AREN'T RELATED TO HORIZONTAL POSITIONING


#SEE IF YOU CAN OVERFIT FOR ONE DATA POINT

#Renormalization!!!!!


#TRY 5x5 in bottleneck class!

#Activations IN BOTH increasing and decreasing of feature maps!!!!!
history = image_reconstruction_64.fit(
    #big_arr.astype('int32')[:, :, :, 0], big_arr.astype('int32')[:, :, :, 0],
    big_arr, big_arr,
    batch_size=32, #INCREASE??????????
    epochs=3000, #MIGHT NEED ALOT OF THIS!!! 
        #Will need to decrease units though
    
    #callbacks=[EarlyStopping(monitor='train_loss')],
    verbose=0
)
pd.DataFrame(history.history)

In [ ]:


MAKE POKEMON FROM NAME!!!

Title -> Image

Random Image -> Pokemon

PCA on Encoding, get 2-dim representations to plot!
Color points for specfic pokemon!! (Squirtle, Wartorle, and Blastoise)

Square the PCA indices in encoding, then DECODE (then try merging just those 2 VALUES)

Divide dataset in half, give two random pokemon, make one random pokemon from OTHER DATASET

layer.Embeddings()




In [ ]:
units = 256
bottleneck_units = 128
restoration_units = 256
regularizer_weight = 0
input_shape = [64, 64, 3]

class TransposeInto2(keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(TransposeInto2, self).__init__()
        
        self.units = units
        
        self.convReduceFilter1 = layers.Conv2D(bottleneck_units, (1, 1), activation=layers.LeakyReLU(), padding="same")
        self.convReduceFilter2 = layers.Conv2D(bottleneck_units, (1, 1), activation=layers.LeakyReLU(), padding="same")
        self.convReduceFilter3 = layers.Conv2D(bottleneck_units, (1, 1), activation=layers.LeakyReLU(), padding="same")
        self.convReduceFilter4 = layers.Conv2D(bottleneck_units, (1, 1), activation=layers.LeakyReLU(), padding="same")
        
        self.FirstImageWidth = layers.Conv2DTranspose(self.units, (3, 1), strides=(2, 1), padding="same", activation=layers.LeakyReLU())
        self.FirstImageHeight = layers.Conv2DTranspose(self.units, (1, 3), strides=(1, 2), padding="same", activation=layers.LeakyReLU())
        
        self.CheckSampling = layers.Conv2D(self.units, (3, 3), activation=layers.LeakyReLU(), padding='same')
        self.CheckSampling2 = layers.Conv2D(self.units, (3, 3), activation=layers.LeakyReLU(), padding='same')
        
        self.BatchNorm3 = layers.BatchNormalization(renorm=batch_renorm)
        self.BatchNorm4 = layers.BatchNormalization(renorm=batch_renorm)
        self.BatchNorm5 = layers.BatchNormalization(renorm=batch_renorm)
        self.BatchNorm6 = layers.BatchNormalization(renorm=batch_renorm)
        self.BatchNorm7 = layers.BatchNormalization(renorm=batch_renorm)
        self.BatchNorm8 = layers.BatchNormalization(renorm=batch_renorm)
        self.BatchNorm9 = layers.BatchNormalization(renorm=batch_renorm)
        self.BatchNorm10 = layers.BatchNormalization(renorm=batch_renorm)
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'units': self.units
        })
        return config
        
        
    def call(self, inputs):
        x = self.convReduceFilter1(inputs)
        x = self.BatchNorm3(x)
        
        x = self.FirstImageWidth(x)
        x = self.BatchNorm4(x)
        
        x = self.convReduceFilter2(x)
        x = self.BatchNorm5(x)
        
        x = self.FirstImageHeight(x)
        x = self.BatchNorm6(x)
        
        x = self.convReduceFilter3(x)
        x = self.BatchNorm7(x)
        x = self.CheckSampling(x)
        x = self.BatchNorm8(x)
        
        x = self.convReduceFilter4(x)
        x = self.BatchNorm9(x)
        x = self.CheckSampling2(x)
        x = self.BatchNorm10(x)
        
        return x
    
class TransposeInto64(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(TransposeInto64, self).__init__()
        inside_units = units*8
        self.TransposeInto2a = TransposeInto2(inside_units)
        self.TransposeInto2b = TransposeInto2(inside_units//2)
        self.TransposeInto2c = TransposeInto2(inside_units//4)
        self.TransposeInto2d = TransposeInto2(inside_units//8)
        self.TransposeInto2e = TransposeInto2(inside_units//16)
        self.TransposeInto2f = TransposeInto2(inside_units//32)
        
        
    def call(self, inputs):
        x = self.TransposeInto2a(inputs)
        x = self.TransposeInto2b(x)
        x = self.TransposeInto2c(x)
        x = self.TransposeInto2d(x)
        x = self.TransposeInto2e(x)
        x = self.TransposeInto2f(x)

        return x


class BottleneckPriorScale(keras.layers.Layer):
    def __init__(self, bottleneck_units, units, regularizer_weight, **kwargs):
        super(BottleneckPriorScale, self).__init__()
        #self.layer1 = layers.Conv2D(bottleneck_units, kernel_size=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding='same') #no activation
        #self.layer2 = layers.Conv2D(units, kernel_size=(3, 3), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")
        #activations after every conv!!!!
        
        self.bottleneck_units = bottleneck_units
        self.units = units
        self.regularizer_weight = regularizer_weight
        
        self.layer5 = layers.Conv2D(self.bottleneck_units, kernel_size=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(self.regularizer_weight), padding="same")
        self.layera = layers.LeakyReLU()
        self.layerb = layers.Dropout(.3)
        self.layerc = layers.BatchNormalization(renorm=batch_renorm)
        
        self.layer8 = layers.Conv2D(self.units, kernel_size=(3, 3), kernel_regularizer=tf.keras.regularizers.l2(self.regularizer_weight), padding="same")
        self.layer9 = layers.LeakyReLU()
        self.layer10 = layers.Dropout(.3)
        self.layer11 = layers.BatchNormalization(renorm=batch_renorm)
        self.layer12 = layers.Conv2D(self.bottleneck_units, kernel_size=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(self.regularizer_weight), padding='same')
        
        self.layerd = layers.LeakyReLU()
        self.layere = layers.Dropout(.3)
        self.layerf = layers.BatchNormalization(renorm=batch_renorm)
        self.layer13 = layers.Conv2D(self.units, kernel_size=(3, 3), kernel_regularizer=tf.keras.regularizers.l2(self.regularizer_weight), padding="same")

        #self.layer16 = layers.Conv2D(bottleneck_units, kernel_size=(1, 3), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")

        #self.layer19 = layers.Conv2D(units, kernel_size=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(regularizer_weight), padding="same")
        self.layer20 = layers.LeakyReLU()
        self.layer21 = layers.Dropout(.3)
        self.layer22 = layers.BatchNormalization(renorm=batch_renorm)
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'bottleneck_units': self.bottleneck_units,
            'units': self.units,
            'regularizer_weight': self.regularizer_weight
        })
        return config

    def call(self, inputs):
        #x = self.layer1(inputs)
        #x = self.layer2(x)

        x = self.layer5(inputs)
        x = self.layera(x)
        x = self.layerb(x)
        x = self.layerc(x)

        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        x = self.layerd(x)
        x = self.layere(x)
        x = self.layerf(x)
        
        x = self.layer13(x)

        #x = self.layer16(x)

        #x = self.layer19(x)
        x = self.layer20(x)
        x = self.layer21(x)
        x = self.layer22(x)
        
        return x

In [ ]:
batch_renorm = False

image_reconstruction_64 = keras.models.load_model("../input/2epochtpumodel/mymodel64.h5",
            custom_objects={'LeakyReLU': layers.LeakyReLU, 'BottleneckPriorScale':BottleneckPriorScale,
                           'TransposeInto64':TransposeInto64, 'TransposeInto2':TransposeInto2})

In [ ]:
%%time
#Don't do compile! Loses either trajectory or all weights!

#loss didn't change (batch size too high or too late?)

history = image_reconstruction_64.fit(
    big_arr, big_arr,
    batch_size=32, 
    epochs=1,
    verbose=0
)
pd.DataFrame(history.history)

In [ ]:
#Random image of a dog, not trained!
size = 64, 64
inp = Image.open('../input/natural-images/natural_images/cat/cat_0001.jpg').resize(size, Image.ANTIALIAS)
print(np.array(inp).shape)
plt.imshow(inp)
plt.show()
out = image_reconstruction_64.predict(np.array(inp).reshape(1, 64, 64, 3))
plt.imshow(out[0].astype('uint8'))

In [ ]:
import tensorflow as tf

all_layer_names = [l.name for l in image_reconstruction_64.layers]

layer_names = ['half', 'decoder_output']
layers = [image_reconstruction_64.get_layer(l).output for l in layer_names]





encoder = tf.keras.Model(image_reconstruction_64.input, layers[0])





idx = all_layer_names.index("half")
input_shape = image_reconstruction_64.layers[idx+1].get_input_shape_at(0)[1:] # get the input shape of desired layer
layer_input = tf.keras.layers.Input(shape=input_shape) # a new input tensor to be able to feed the desired layer
print(input_shape)

# create the new nodes for each layer in the path
x = layer_input
for layer in image_reconstruction_64.layers[idx+1:]:
    x = layer(x)

# create the model

decoder = tf.keras.Model(layer_input, x)

In [ ]:
encodings = encoder.predict(big_arr)

In [ ]:
enc_arr = np.array([e[0][0] for e in encodings])
enc_arr.shape

In [ ]:
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot

X, y = make_regression(n_samples=len(big_arr), n_features=len(enc_arr[0]), n_informative=5, random_state=1)

model = DecisionTreeRegressor()

model.fit(X, y)

importance = model.feature_importances_

for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
#From here: https://stats.stackexchange.com/questions/108743/methods-in-r-or-python-to-perform-feature-selection-in-unsupervised-learning
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler

class PFA(object):
    def __init__(self, n_features, q=None):
        self.q = q
        self.n_features = n_features

    def fit(self, X):
        if not self.q:
            self.q = X.shape[1]

        sc = StandardScaler()
        X = sc.fit_transform(X)

        pca = PCA(n_components=self.q).fit(X)
        A_q = pca.components_.T

        kmeans = KMeans(n_clusters=self.n_features).fit(A_q)
        clusters = kmeans.predict(A_q)
        cluster_centers = kmeans.cluster_centers_

        dists = defaultdict(list)
        for i, c in enumerate(clusters):
            dist = euclidean_distances([A_q[i, :]], [cluster_centers[c, :]])[0][0]
            dists[c].append((i, dist))

        self.indices_ = [sorted(f, key=lambda x: x[1])[0][0] for f in dists.values()]
        self.features_ = X[:, self.indices_]

In [ ]:
import numpy as np

pfa = PFA(n_features=2, q=0.95)
pfa.fit(enc_arr)

# To get the transformed matrix
X = pfa.features_

# To get the column indices of the kept features
column_indices = pfa.indices_
column_indices

In [ ]:
import numpy as np

pfa = PFA(n_features=2)
pfa.fit(enc_arr.T)

# To get the transformed matrix
X = pfa.features_

# To get the column indices of the kept features
row_indices = pfa.indices_
row_indices

In [ ]:
#Most important pictures
plt.imshow(big_arr[row_indices[0]].astype('uint8'))

In [ ]:
plt.imshow(big_arr[row_indices[1]].astype('uint8'))

In [ ]:

#K-Means is non deterministics, I MAY NEED TO RUN IT A COUPLE OF TIMES

for e in enc_arr:
    e[column_indices[0]] = e[column_indices[0]]**2
    e[column_indices[1]] = e[column_indices[1]]**2

In [ ]:
changed_images = decoder.predict(enc_arr.reshape(-1, 1, 1, 4096))
plt.imshow(changed_images[0].astype('uint8'))

In [ ]:
enc_arr = np.array([e[0][0] for e in encodings])
enc_arr[0][55:200] = 1000
enc_arr[0][column_indices[1]] = 77
changed_images = decoder.predict(enc_arr[0].reshape(-1, 1, 1, 4096))
plt.imshow(changed_images[0].astype('uint8'))

In [ ]:
#plt.imshow(big_arr[27])
#plt.show()
#plt.imshow(big_arr[47])
#plt.show()
plt.imshow(big_arr[30])
plt.show()
plt.imshow(big_arr[150])
plt.show()
encodings = encoder.predict(np.array([big_arr[30], big_arr[150]]))
print(encodings.shape)
#merged = ((encodings[0]+encodings[1]+encodings[2]+encodings[3])/4)
merged = ((encodings[0]*5+encodings[1])/6)
pred = decoder.predict(merged.reshape(1, 1, 1, -1))

x_min = pred[0].min(axis=(1, 2), keepdims=True)
x_max = pred[0].max(axis=(1, 2), keepdims=True)

p = (pred[0] - x_min)/(x_max-x_min)
plt.imshow(p*1.2)

In [ ]:
cubone_and_marok = [6, 7]
eggs_and_tor = [4, 5]
color_arr = []
plt.imshow(big_arr[4])
plt.show()
plt.imshow(big_arr[5])
plt.show()
for i in range(len(dim_reduced)):
    if i in cubone_and_marok:
        color_arr.append("Orange")
    elif i in eggs_and_tor:
        color_arr.append("Green")
    else:
        color_arr.append("Blue")
        
plt.scatter(dim_reduced, color=color_arr)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LeakyReLU
import tensorflow as tf

latent_dim = 1000

input_shape = [256, 256, 1]
input_layer = layers.Input(input_shape)
conv_layer1 = layers.Conv2D(1000, kernel_size=(256, 1))(input_layer)
conv_layer2 = layers.Conv2D(1000, kernel_size=(1, 256), activation=LeakyReLU())(conv_layer1) 
batch_norm1 = layers.BatchNormalization()(conv_layer2)
mean_layer = layers.Dense(1000, name="mean_output")(batch_norm1)
std_layer = layers.Dense(1000, name="std_output")(batch_norm1)

from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([mean_layer, std_layer])

#encoder = Model.model(input_layer, [mean_layer, std_layer, z])
#would need to do encoder[2]
encoder = Model.model(input_layer, z)


input_layer2 = layers.Input(1000)
dense_layer = layers.Dense(1000, activation='relu')
dense_layer = layers.Dense(256*256*1, activation='sigmoid')
output = layers.Reshape(input_shape, name="decoder_output")

decoder = Model.mode(input_layer2, output)


whole_output = decoder(encoder(input_layer))
var_autoencoder = Model.model(input_layer, whole_output, name="variantional")

var_autoencoder.compile(loss=reconstruction_loss_func,
                   optimizer=tf.keras.optimizers.Adam(lr=0.0005))
                   #metrics=['accuracy'])

#Do I need validation data????
history = var_autoencoder.fit(
    big_arr[0].reshape(1, 64, 64, 1), big_arr[0].reshape(1, 64, 64, 1),
    epochs=100,
    batch_size = 1,
    verbose=0
)
pd.DataFrame(history.history)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LeakyReLU
import tensorflow as tf

#Bad Accuracy BUTTTTTTT
#TRY OUT THIS LATENT SPACE FIRST!

def reconstruction_loss_func(old, new):
    #1 for each wrong pixel, harsh!
    
    reconstruction_loss = keras.losses.binary_crossentropy(old, new)
    reconstruction_loss *= (256*256*3)
    
    return reconstruction_loss

latent_dim = 1024
input_shape = [64, 64, 3]
autoencoder = keras.Sequential([
    #layers.Flatten(input_shape=input_shape),
    layers.BatchNormalization(input_shape=input_shape, renorm=True),
    layers.Conv2D(latent_dim, kernel_size=(64, 1), input_shape=input_shape),
    layers.Conv2D(latent_dim, kernel_size=(1, 64)),
    layers.Dense(2048, activation=layers.LeakyReLU(), name="encoder_output"),
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(latent_dim, kernel_size=(1, 1)),
    layers.Dense(64*64*3),
    layers.LeakyReLU(),
    layers.Reshape(input_shape, name="decoder_output")
])

#Is a direct mapping to the SHAPE I want better???

autoencoder.compile(loss='mse',
                   optimizer=tf.keras.optimizers.Adam(lr=0.005, clipnorm=1.0, clipvalue=.5),
                   metrics=['accuracy'])

#Do I need validation data????
history = autoencoder.fit(
    big_arr, big_arr,
    epochs=1500,
    batch_size=32,
    verbose=0
)
pd.DataFrame(history.history)

In [ ]:
image_reconstruction.layers[30].name

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def show_image(arr):
    w, h = 28, 28
    img = Image.fromarray(arr, 'L')
    img.save('my.png')
    img.show()
    plt.imshow(arr, cmap='gray')
    
    return img

In [ ]:
ex_image = big_arr[55]
prediction = autoencoder.predict(np.array([ex_image]).astype('int32'))

print(ex_image.dtype)

plt.imshow(ex_image)

In [ ]:
print(np.unique(prediction[0]))

plt.imshow(prediction[0].astype('uint8'))

In [ ]:
## WHOA
import tensorflow as tf


#.predict() returns data
#.output() RETURNS A LAYER

all_layer_names = [l.name for l in autoencoder.layers]

layer_names = ['encoder_output', 'decoder_output']
layers = [autoencoder.get_layer(l).output for l in layer_names]

encoder = tf.keras.Model(autoencoder.input, layers[0])





idx = all_layer_names.index("encoder_output")
input_shape = autoencoder.layers[idx].get_input_shape_at(0)[1:] # get the input shape of desired layer
layer_input = tf.keras.layers.Input(shape=input_shape) # a new input tensor to be able to feed the desired layer
print(input_shape)

# create the new nodes for each layer in the path
x = layer_input
for layer in autoencoder.layers[idx:]:
    x = layer(x)

# create the model

decoder = tf.keras.Model(layer_input, x)

In [ ]:
ex_image = big_arr[44]
prediction = image_reconstruction_64.predict(np.array([ex_image]).astype('int32'))

print(ex_image.dtype)

plt.imshow(ex_image)

In [ ]:
print(np.unique(prediction[0]))
plt.imshow(prediction[0].astype('uint8'))

## get the layer names!
image_reconstruction.summary()

In [ ]:
## WHOA
import tensorflow as tf


#.predict() returns data
#.output() RETURNS A LAYER

all_layer_names = [l.name for l in image_reconstruction_64.layers]

layer_names = ['half', 'decoder_output']
layers = [image_reconstruction_64.get_layer(l).output for l in layer_names]

encoder = tf.keras.Model(image_reconstruction_64.input, layers[0])





idx = all_layer_names.index("half")
input_shape = image_reconstruction_64.layers[idx+1].get_input_shape_at(0)[1:] # get the input shape of desired layer
layer_input = tf.keras.layers.Input(shape=input_shape) # a new input tensor to be able to feed the desired layer
print(input_shape)

# create the new nodes for each layer in the path
x = layer_input
for layer in image_reconstruction_64.layers[idx+1:]:
    x = layer(x)

# create the model

decoder = tf.keras.Model(layer_input, x)

In [ ]:
ex_image = big_arr[801]
prediction = image_reconstruction_64.predict(np.array([ex_image]))
plt.imshow(ex_image)

In [ ]:
plt.imshow(prediction.astype('uint8')[0])

In [ ]:
ex_image = big_arr[432]
vec1 = encoder.predict(np.array([ex_image]))
ex_image = big_arr[801]
vec2 = encoder.predict(np.array([ex_image]))
vec = ((vec1+vec2))/2

print(vec1==vec2)
print(vec1)
print(vec2)

changed_image = decoder.predict(vec)
plt.imshow(changed_image[0].astype('uint8'))

In [ ]:
#NOW READ!!!

image_into_df
.get_layer().output
model.Model(, [outputs_LIST])
output = model()
tf.gradient_tape as tape:
    compute_loss()
grads = tape.gradient(loss, image)
opt.apply_gradients([grads, init_image])
    
def get_functional_model_and_layers():
    layers_style = ['block3_conv2']
    layers_content = ['block6_conv1', 'block7_conv1']
    
    vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    
    outputs = [vgg.get_layer(ls).output for ls in layers_style]+ \
            [vgg.get_layer(lc).output for lc in layers_content]

    return Model.model(vgg.input, outputs)
    
def content_loss(base_content, target)
    tf.reduce_mean(tf.square(base_content-target))
    
def style_loss(base_style, gram_target):
    tf.reduce_mean(tf.square(tf.subtract(gram_matrix(base_style), gram_target)))

def gram_matrix(A):
    channels = A.shape[-1]
    a = A.reshape(-1, channels)
    n = A.shape[0]
    
    return tf.matmul(a, a, transpose=True) / n.cast(float32)
    

def feature_representation(style_path, content_path):
    im = load_and_process_img(style_path)
    im2 = load_and_process_img(content_path)
    
    inp = np.concantanate([im, im2])
    
    output = model(inp)
    
    #one for each layer
    return [output[0]], [output[1], output[2]]
    
    
def combine_losses():
    
    
def compute_grad(cfg):
    with tf.gradientTape() as tape:
        loss = combine_losses(**cfg)
    total_loss = loss[0]
    better_image = tape.gradient(all_loses, init_image) #!!!!!!!!!
    
def run():
    
    
run()

In [ ]:
#TWO INPUTS TO DISCRIMINATOR, So I could train discriminator as an anchor (anchor input too)
    #Give it two images, it must determine if they are in the same class
    #Sometimes give it 2 real images, sometimes give it a fake image so discriminator gets
        #doesn't just start guessing all outputs are fake
    
#DISCRIMINATOR INPUT EXAMPLE:
    #text, image of same class(fake/real)
    #image, image of same class(fake/real)
    #image, same image at night(different fake image at night/real)
    
#Conditional discriminator makes for: CLASS SPECFIC GENERATOR

#FOR GANS TO WORK, YOU NEED A DETAILED & WIDE DATASET OF "REAL" IMAGES

vgg = tf.applications.keras.vgg16()
model = Model.model(vgg.input, vgg.get_layer('block5_conv1').output)
model_discriminator = keras.Model([
        layers.BatchNormalization(input_shape=input_shape),
        layers.Dense(1)
])
def gan(samples, batch_size=5):
    
    #MSE is high when image is way off, 
            #score of discrimator might not do that
    #i can't use mse, but MSE correlates best to getting the image right???
    
    #MAYBE IF WE DID ENOUGH SCORING DURING EVERY ITERATION IT WOULD EQUAL AN MSE
    
    #model_discriminator.compile()
    
    for sample in samples.shuffle(batch_size):
        fake_image = model(sample)
        pred = model_discriminator.predict(fake_image)
        gen_loss = 0
        dis_loss = 0
        if pred != 0:
            dis_loss = 1
        else:
            gen_loss = 1
    
    with tf.gradientTape() as tape:
        loss = tf.reduce_mean(tf.square(target-model(sample.shuffle(1))))
    grads = tape.gradient(loss, model.get_weights())
    #grads = tape.gradient(gen_loss, model.get_weights())
    #grads = tape.gradient(dis_loss, model_discriminator.get_weights())
    opt = Adam.optimizer(loss='mse')
    new_weights = model.get_weights()
    opt.apply_gradients([grads, new_weights])
    model.assign_weights(new_weights)

In [ ]:
import tensorflow as tf
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

In [ ]:
#minimzing distance between intermediate values
#Optimizing hidden values??

In [ ]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer we are interested in
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

In [ ]:
def get_model():
  """ Creates our model with access to intermediate layers. 
  
  This function will load the VGG19 model and access the intermediate layers. 
  These layers will then be used to create a new model that will take input image
  and return the outputs from these intermediate layers from the VGG model. 
  
  Returns:
    returns a keras model that takes image inputs and outputs the style and 
      content intermediate layers. 
  """
  # Load our model. We load pretrained VGG, trained on imagenet data (weights=’imagenet’)
  vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  # Get output layers corresponding to style and content layers 
  style_outputs = [vgg.get_layer(name).output for name in style_layers]
  content_outputs = [vgg.get_layer(name).output for name in content_layers]
  model_outputs = style_outputs + content_outputs
  # Build model 
  return models.Model(vgg.input, model_outputs)

In [ ]:
#Do the same thing for:
    #1. Textual Data
    #2. My own model
    #3. Different layers, sharing or not sharing a loss function
    
    #vgg.get_layer(name).output
    #name=""

In [ ]:
#simple content loss function, MSE

def get_content_loss(base_content, target):
  return tf.reduce_mean(tf.square(base_content - target))

In [ ]:
#This is a complicated gram_matrix for style loss
#Apparently it's just a kernel feature map/receptive field
    #MSE at the end though

def gram_matrix(input_tensor):
  # We make the image channels first 
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a=True)
  return gram / tf.cast(n, tf.float32)

#gram matrix, a square matrix
    #matrix reshape to [LxW, Channels]
    #Squared itself, Channels get dot producted out of existence
    #divide [LxW, LxW] by LxW
 
def get_style_loss(base_style, gram_target):
  """Expects two images of dimension h, w, c"""
  # height, width, num filters of each layer
  height, width, channels = base_style.get_shape().as_list()
  gram_style = gram_matrix(base_style)
  
  return tf.reduce_mean(tf.square(gram_style - gram_target))

In [ ]:
def get_feature_representations(model, content_path, style_path):
  """Helper function to compute our content and style feature representations.
 
  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image
    
  Returns:
    returns the style features and the content features. 
  """
  # Load our images in 
  content_image = load_and_process_img(content_path)
  style_image = load_and_process_img(style_path)
  
  # batch compute content and style features
  stack_images = np.concatenate([style_image, content_image], axis=0)
  model_outputs = model(stack_images)
  
  # Get the style and content feature representations from our model  
  style_features = [style_layer[0] for style_layer in model_outputs[:num_style_layers]]
  content_features = [content_layer[1] for content_layer in model_outputs[num_style_layers:]]
  return style_features, content_features

In [ ]:
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
  """This function will compute the loss total loss.
  
  Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
  Returns:
    returns the total loss, style loss, content loss, and total variational loss
  """
  style_weight, content_weight, total_variation_weight = loss_weights
  
  # Feed our init image through our model. This will give us the content and 
  # style representations at our desired layers. Since we're using eager
  # our model is callable just like any other function!
  model_outputs = model(init_image)
  
  style_output_features = model_outputs[:num_style_layers]
  content_output_features = model_outputs[num_style_layers:]
  
  style_score = 0
  content_score = 0

  # Accumulate style losses from all layers
  # Here, we equally weight each contribution of each loss layer
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, style_output_features):
    style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
    
  # Accumulate content losses from all layers 
  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, content_output_features):
    content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
  style_score *= style_weight
  content_score *= content_weight
  total_variation_score = total_variation_weight * total_variation_loss(init_image)

  # Get total loss
  loss = style_score + content_score + total_variation_score 
  return loss, style_score, content_score, total_variation_score

In [ ]:
def compute_grads(cfg):
  with tf.GradientTape() as tape: 
    all_loss = compute_loss(**cfg)
  # Compute gradients wrt input image
  total_loss = all_loss[0]
  return tape.gradient(total_loss, cfg['init_image']), all_loss

#our loss function with respect to our input image for the backwards pass.

In [ ]:
def run_style_transfer(content_path, 
                       style_path,
                       num_iterations=1000,
                       content_weight=1e3, 
                       style_weight = 1e-2): 
  display_num = 100
  # We don't need to (or want to) train any layers of our model, so we set their trainability
  # to false. 
  model = get_model() 
  for layer in model.layers:
    layer.trainable = False
  
  # Get the style and content feature representations (from our specified intermediate layers) 
  style_features, content_features = get_feature_representations(model, content_path, style_path)
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
  
  # Set initial image
  init_image = load_and_process_img(content_path)
  init_image = tfe.Variable(init_image, dtype=tf.float32)
  # Create our optimizer
  opt = tf.train.AdamOptimizer(learning_rate=10.0)

  # For displaying intermediate images 
  iter_count = 1
  
  # Store our best result
  best_loss, best_img = float('inf'), None
  
  # Create a nice config 
  loss_weights = (style_weight, content_weight)
  cfg = {
      'model': model,
      'loss_weights': loss_weights,
      'init_image': init_image,
      'gram_style_features': gram_style_features,
      'content_features': content_features
  }
    
  # For displaying
  plt.figure(figsize=(15, 15))
  num_rows = (num_iterations / display_num) // 5
  start_time = time.time()
  global_start = time.time()
  
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means   
  for i in range(num_iterations):
    grads, all_loss = compute_grads(cfg)
    loss, style_score, content_score = all_loss
    # grads, _ = tf.clip_by_global_norm(grads, 5.0)
    opt.apply_gradients([(grads, init_image)])
    clipped = tf.clip_by_value(init_image, min_vals, max_vals)
    init_image.assign(clipped)
    end_time = time.time() 
    
    if loss < best_loss:
      # Update best loss and best image from total loss. 
      best_loss = loss
      best_img = init_image.numpy()


      iter_count += 1
  print('Total time: {:.4f}s'.format(time.time() - global_start))
      
  return best_img, best_loss 

In [ ]:
#No batch normalization until after!
from keras import backend as K

latent_dim = 1024
class encoding_layer():
    def __init__():
        super(encoding_layer, self)
        self.embeddDense = layers.Dense(latent_dim*2, activation=layers.LeakyReLU())
        self.MeanNoAct = layers.Dense(latent_dim)
        self.StdNoAct = layers.Dense(latent_dim)
        self.Forget = layers.Dense(latent_dim, activation='softmax')

    def call(inputs):
        x = embeddDense(inputs)
        m = self.MeanNoAct(x)
        s = self.StdNoAct(x)
        f = self.Forget(x)
        
        return m, s, f

def sampling(inputs):
    mean, std, forget = inputs
    epilson = K.random.normal(shape=(K.shape(mean)[0]), latent_dim, mean=0, stdev=.1)
        
    return (mean + K.exp(std) * eplison) * forget
    
class decoding_layer():
    def __init__():
        super(decoding_layer, self)
        self.lamb = layers.Lambda(sampling)
        
    def call(inputs)
        x = self.lamb(inputs)
        
        return x
    
   

class expand:
    def __init__(self, units):
        super(expand, self)
        self.bottle = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
        self.norm1 = layers.BatchNormalization(renorm=batch_renorm)   
        self.trans1 = layers.Conv2DTranspose(512, (3, 1), strides=(2, 1), padding='same', activation=layers.LeakyReLU())
        self.norm2 = layers.BatchNormalization(renorm=batch_renorm)
        self.trans2 = layers.Conv2DTranspose(512, (1, 3), strides=(1, 2), padding='same', activation=layers.LeakyReLU())
        self.norm3 = layers.BatchNormalization(renorm=batch_renorm)
        self.conv1 = layers.Conv2D(512, (3, 1), padding='same', activation=layers.LeakyReLU())
        self.norm4 = layers.BatchNormalization(renorm=batch_renorm)
        self.conv2 = layers.Conv2D(512, (1, 3), padding='same', activation=layers.LeakyReLU())
        self.norm5 = layers.BatchNormalization(renorm=batch_renorm)
        
    def call(inputs):
        x = self.bottle(inputs)
        x = self.norm1(x)
        x = self.trans1(x)
        x = self.norm2(x)
        x = self.trans2(x)
        x = self.norm3(x)
        x = self.conv1(x)
        x = self.norm4(x)
        x = self.conv2(x)
        x = self.norm5(x)
        
        return x

x = layers.Input(input_shape=[100])(inputs)
x = layers.Dense(1024, activation=layers.LeakyReLU())(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
enc_outputs = encoding_layer()(x)
x = decoding_layer()(enc_outputs)
x = expand(512)(x)
x = expand(512)(x)
x = expand(256)(x)
x = expand(128)(x)
x = expand(64)(x)
x = expand(32)(x)
x = layers.Conv2D(3, (3, 3), padding="same", activation=layers.LeakyReLU())
generator_model = keras.Model(inputs, x)

inputs = big_arr?????

x = layers.Input(input_shape=[64, 64, 3])(inputs)
x = layers.Conv2D(64, (3, 3), strides=(2, 2), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (3, 3), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(128, (3, 3), strides=(2, 2), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(128, (3, 3), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(128, (3, 3), strides=(2, 2), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(256, (3, 3), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(256, (3, 3), strides=(2, 2), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(256, (3, 3), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(256, (3, 3), strides=(2, 2), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(512, (3, 3), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(512, (3, 3), strides=(2, 2), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(512, (3, 3), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(512, (3, 3), strides=(2, 2), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(512, (3, 3), activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(64, (1, 1), padding='same', activation=layers.LeakyReLU())
x = layers.Dropout(.4)(x)
x = layers.BatchNormalization(renorm=batch_renorm)(x)
x = layers.Conv2D(1, (1, 1), padding='same', activation='sigmoid')
discrim_model = keras.Model(inputs, x)
    
def reconstruction_loss_func(old, new):
    #1 for each wrong pixel, harsh!
    
    #reconstruction_loss = keras.losses.binary_crossentropy(tf.bitcast(tf.cast(old, tf.int8), tf.uint8), tf.bitcast(tf.cast(new, tf.int8), tf.uint8))
    mean, sigma, _ = enc_outputs
    k1_loss = 1 + sigma - K.square(mean) - K.exp(sigma)
    k1_loss = K.sum(k1_loss, axis=-1)
    k1_loss *= -.5
    
    reconstruction_loss = keras.losses.binary_crossentropy(old, new)
    reconstruction_loss *= (256*256*3)
    
    return reconstruction_loss + k1_loss


generator_model.compile(
`#switch between rec loss and discriminator loss!
    #loss='binary_crossentropy', 
    loss = reconstruction_loss_func,
    optimizer=keras.optimizer.Adam(lr=0.0002, beta_1=0.5),
    metrics=['accuracy']
)
history = image_reconstruction_64.fit(
    big_arr, big_arr,
    batch_size=32, 
    epochs=100,
    verbose=0
)

Inputs: Point in latent space, e.g. a 100 element vector of Gaussian random numbers.

In [ ]:
lr=0.0002, beta_1=0.5
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	x_input = randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

def generate_fake_samples(g_model, latent_dim, n_samples):
	x_input = generate_latent_points(latent_dim, n_samples)
	X = g_model.predict(x_input)
	y = zeros((n_samples, 1))
	return X, y